<a href="https://colab.research.google.com/github/kazimianec/TARIC/blob/master/TRCtransformersTF_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np

In [ ]:
!pip install tensorflow_text
import tensorflow_text

In [5]:
url_TARIC_C2_descrptions = "https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C2_descriptions.csv"
url_TARIC_C4_descrptions ="https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C4_descriptions.csv"
url_TARIC_C6_descrptions ="https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C6_descriptions.csv"
url_TARIC_C246_descrptions = "https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C246_descriptions_CONCAT.csv"


In [6]:
import pandas as pd 
import io 
  
df_C2_descriptions = pd.read_csv(url_TARIC_C2_descrptions) 
df_C4_descriptions = pd.read_csv(url_TARIC_C4_descrptions)
df_C6_descriptions = pd.read_csv(url_TARIC_C6_descrptions)
df_C246_descriptions = pd.read_csv(url_TARIC_C246_descrptions)

In [7]:
for col in df_C246_descriptions.columns: 
    print(col)

CODE_2
CODE_4
CODE_6
goods_CODE_6
Description_46
Description_246
Description_46_concat
Description_246_concat


In [8]:
#sentences = df['CLASS_Description'].tolist()

sentences_C2 = df_C2_descriptions['Description_2'].tolist()
sentences_C4 = df_C4_descriptions['Description_4'].tolist()
sentences_C6 = df_C6_descriptions['Description_6'].tolist()

sentences_C246 = df_C246_descriptions['Description_246'].tolist()
sentences_C46 = df_C246_descriptions['Description_46'].tolist()

sentences_C246_concat = df_C246_descriptions['Description_246_concat'].tolist()
sentences_C46_concat = df_C246_descriptions['Description_46_concat'].tolist()

In [9]:
sentences = sentences_C46
sentences = [[element.lower()] for element in sentences]
TARIC_sentences = tf.squeeze(tf.constant(sentences))


**TEST**

In [ ]:
sentences

**END of TEST** **sentences** **bold text** **bold text**

**Test 0. "Good descriptions"**

**Test 1. English, all words are NOT in TARIC**

**Test 2. Englsih, words are in "EXCLUDING ...", "OTHER THAN...." parts of TARIC descriptions**

**Test 3. Not obvious descriptions**

In [13]:
description = ["Sterling Silver Turquoise Art Deco Earrings"]
description = ["I Love Makeup - Nail Polish"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]

**ENCODER BERT**

In [11]:
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
#preprocessor = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")


In [121]:
preprocessor.trainable

False

In [12]:
#encoder = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3")
#encoder = hub.load("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1")

#encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/en-base/1")



In [13]:
encoder.trainable

False

In [14]:
prep = preprocessor(TARIC_sentences)
print(prep.keys())

dict_keys(['input_word_ids', 'input_mask', 'input_type_ids'])


In [15]:
prep["input_mask"]

<tf.Tensor: shape=(5209, 128), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>

In [ ]:
#embeddings_TARIC_long_dict = encoder(prep)
# returns a dictionary with keys dict_keys(['encoder_outputs', 'sequence_output', 'default', 'pooled_output'])
#print(embeddings_TARIC_long.keys())

In [16]:
print(embeddings_TARIC_long_dict["encoder_outputs"][0].shape)
print(embeddings_TARIC_long_dict["sequence_output"].shape)
print(embeddings_TARIC_long_dict["sequence_output"].shape)
print(embeddings_TARIC_long_dict["default"].shape)
print(embeddings_TARIC_long_dict["pooled_output"].shape)

NameError: ignored

In [108]:
print(embeddings_TARIC_long_dict.keys())

dict_keys(['sequence_output', 'default', 'pooled_output', 'encoder_outputs'])


In [109]:
embeddings_TARIC_default = embeddings_TARIC_long_dict["default"]
embeddings_TARIC_encoder_outputs = embeddings_TARIC_long_dict["encoder_outputs"]
embeddings_TARIC_sequence_output = embeddings_TARIC_long_dict["sequence_output"]
embeddings_TARIC_pooled_output = embeddings_TARIC_long_dict["pooled_output"]

In [ ]:
# cycle is less efficient comparing to vectorised processing, but uses less memory
# and vector version would be just:
#embeddings_TARIC_long = encoder(prep)["default"]
#embeddings_TARIC_long = encoder(preprocessor(TARIC_sentences))
embeddings_TARIC_long = []

# take a care on square brackets. It expects a tensor, not just a string. So, reshaping sentence_id
for sentence_id in sentences:
  print(sentence_id)
  embeddings_TARIC_id = encoder(preprocessor(sentence_id))["default"]

In [ ]:
                               
                                
# embeddings_TARIC_long.append(embeddings_TARIC_id)

In [ ]:
#embeddings_TARIC_long = tf.squeeze(embeddings_TARIC_long)
print(embeddings_TARIC_long.shape)

(98, 768)


In [ ]:
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-lite/2")
#embed = hub.load ("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3")


In [ ]:
# = embed(sentences)

In [ ]:


description = ["Sterling Silver Turquoise Art Deco Earrings"]




description = ["Vyriškos kelnės"]

description = ["I Love Makeup - Nail Polish"] #yes!
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"] #no
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["Fosmon Apple iPhone 4 / iPhone 4G USB Sync Charge Data Cable with USB Auto Car Charger and USB Home Travel Charger"] #SUPER matches S3
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"] # not


description = ["HTC HD7 Crowned Heart Phone Protector Cover Case"] ## bullshit is both cases - bert and S3
description = ["Fosmon Apple iPhone 4 / iPhone 4G USB Sync Charge Data Cable with USB Auto Car Charger and USB Home Travel Charger"] #SUPER matches S3 

description = ["Sterling Silver Turquoise Art Deco Earrings"]


description = ["Fosmon Apple iPhone 4 / iPhone 4G USB Sync Charge Data Cable with USB Auto Car Charger and USB Home Travel Charger"] #SUPER matches S3 



description = ["Jolly Jumper Pashmama Nursing Cover - Tan "]
description = ["HTC HD7 Crowned Heart Phone Protector Cover Case"]





description = ["Jolly Jumper Pashmama Nursing Cover - Tan"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]

description = ["Fosmon Apple iPhone 4 / iPhone 4G USB Sync Charge Data Cable with USB Auto Car Charger and USB Home Travel Charger"]

description = ["Meat beef"]
description = ["I Love Makeup - Nail Polish"] #yes!

TARIC_description = tf.constant(description)
embeddings_description = encoder(preprocessor(TARIC_description))["default"]


cosine=np.dot(embeddings_TARIC_long, np.transpose(embeddings_description))
cosine=np.inner(embeddings_TARIC_long, embeddings_description)
#print(cosine)
ind_max = tf.constant(tf.argmax(cosine)).numpy()[0]
top_ind = tf.math.top_k(np.transpose(cosine), k=5, sorted=True, name=None).indices.numpy()[0]
top_value = tf.math.top_k(np.transpose(cosine), k=5, sorted=True, name=None).values.numpy()[0]
print(cosine[ind_max])
#print(classes_6[ind_max])
print(top_value[0], sentences[top_ind[0]])
print(top_value[1], sentences[top_ind[1]])
print(top_value[2], sentences[top_ind[2]])
print(top_value[3], sentences[top_ind[3]])
print(top_value[4], sentences[top_ind[4]])

[310.293]
310.293 carpets and other textile floor coverings
310.1679 photographic or cinematographic goods
308.61658 fertilisers
307.73706 pharmaceutical products
307.32385 man-made staple fibres


In [ ]:
cosine.shape

(98, 1)